In [1]:
import os
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine,inspect

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy
import csv,sqlite3


In [2]:
con = sqlite3.connect("./db/flights_data.sqlite")


In [3]:
df = pd.read_csv("./db/flights_metadata.csv")
df.head(1)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
0,2013,1,9E,Pinnacle Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",139.0,37.0,13.04,0.8,...,9.24,2.0,5.0,1860.0,789.0,75.0,451.0,0.0,545.0,NaN


In [4]:
del df['Unnamed: 21']

In [5]:
df.head(1)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2013,1,9E,Pinnacle Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",139.0,37.0,13.04,0.8,...,0.0,9.24,2.0,5.0,1860.0,789.0,75.0,451.0,0.0,545.0


In [6]:
df = df.rename(columns = {" month":"month"," weather_ct":"weather_ct"," arr_delay":"arr_delay"," carrier_delay":"carrier_delay"})

In [7]:
df = df[["year","month","carrier","carrier_name","airport","airport_name","arr_flights","arr_del15","carrier_ct","weather_ct",
"nas_ct","security_ct","late_aircraft_ct","arr_cancelled","arr_diverted","arr_delay","carrier_delay",
"weather_delay","nas_delay","security_delay","late_aircraft_delay"]]

In [8]:
df.head(1)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2013,1,9E,Pinnacle Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",139.0,37.0,13.04,0.8,...,0.0,9.24,2.0,5.0,1860.0,789.0,75.0,451.0,0.0,545.0


In [9]:
df1 = pd.read_csv("./db/airport_codes.csv")
df1.head()

,locationID,Latitude,Longitude
0,ADK,51.8781,176.6461
1,AKK,56.9386,154.1825
2,Z13,60.9047,161.4225
3,AKI,60.9028,161.2306
4,AUK,62.6800,164.6600


In [10]:
df1 = df1.rename(columns = {"locationID":"airport"})
df1.head()


,airport,Latitude,Longitude
0,ADK,51.8781,176.6461
1,AKK,56.9386,154.1825
2,Z13,60.9047,161.4225
3,AKI,60.9028,161.2306
4,AUK,62.6800,164.6600


In [11]:
df2 = df.merge(df1)
# df2 = df.merge(df1,left_on = 'airport',right_on = 'locationID')
df2.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Latitude,Longitude
0,2013,1,9E,Pinnacle Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",139.0,37.0,13.04,0.80,...,2.0,5.0,1860.0,789.0,75.0,451.0,0.0,545.0,32.8969,97.0381
1,2013,1,AA,American Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",13125.0,2372.0,470.16,98.23,...,338.0,35.0,129258.0,34386.0,9641.0,35490.0,191.0,49550.0,32.8969,97.0381
2,2013,1,AS,Alaska Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",62.0,16.0,3.39,0.00,...,0.0,0.0,528.0,206.0,0.0,309.0,0.0,13.0,32.8969,97.0381
3,2013,1,B6,JetBlue Airways,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",89.0,34.0,4.67,0.00,...,0.0,1.0,1617.0,396.0,0.0,1077.0,0.0,144.0,32.8969,97.0381
4,2013,1,DL,Delta Air Lines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",415.0,55.0,22.03,2.31,...,4.0,2.0,2912.0,1110.0,281.0,685.0,0.0,836.0,32.8969,97.0381


In [12]:
df2.to_sql("flights_data", con =con, if_exists='replace', index=False)


In [13]:
df2.dtypes

year                     int64
month                    int64
carrier                 object
carrier_name            object
airport                 object
airport_name            object
arr_flights            float64
arr_del15              float64
carrier_ct             float64
weather_ct             float64
nas_ct                 float64
security_ct            float64
late_aircraft_ct       float64
arr_cancelled          float64
arr_diverted           float64
arr_delay              float64
carrier_delay          float64
weather_delay          float64
nas_delay              float64
security_delay         float64
late_aircraft_delay    float64
Latitude               float64
Longitude              float64
dtype: object

In [14]:
engine = create_engine("sqlite:///db/flights_data.sqlite", encoding='utf8')
conn = engine.connect()

In [15]:
inspector = inspect(engine)
inspector.get_table_names()
inspector.get_columns("flights_data")


[{'name': 'year',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'month',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'carrier',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'carrier_name',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'airport',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'airport_name',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'arr_flights',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'arr_del15',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': 'aut

In [16]:
session = Session(engine)

In [17]:
flight_data = pd.read_sql("SELECT * FROM flights_data",engine)
flight_data.head(2)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Latitude,Longitude
0,2013,1,9E,Pinnacle Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",139.0,37.0,13.04,0.80,...,2.0,5.0,1860.0,789.0,75.0,451.0,0.0,545.0,32.8969,97.0381
1,2013,1,AA,American Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",13125.0,2372.0,470.16,98.23,...,338.0,35.0,129258.0,34386.0,9641.0,35490.0,191.0,49550.0,32.8969,97.0381


In [18]:
flight_data.dtypes

year                     int64
month                    int64
carrier                 object
carrier_name            object
airport                 object
airport_name            object
arr_flights            float64
arr_del15              float64
carrier_ct             float64
weather_ct             float64
nas_ct                 float64
security_ct            float64
late_aircraft_ct       float64
arr_cancelled          float64
arr_diverted           float64
arr_delay              float64
carrier_delay          float64
weather_delay          float64
nas_delay              float64
security_delay         float64
late_aircraft_delay    float64
Latitude               float64
Longitude              float64
dtype: object

In [103]:
airports_data = pd.read_sql("SELECT airport_name,arr_flights,month,Latitude,Longitude FROM flights_data WHERE airport IN ('ATL', 'DFW', 'SFO', 'ORD', 'DEN', 'LAX', 'PHX', 'HOU', 'LAS', 'MSP')",engine)
#     month = int(month)
#     test_data = airports_data.loc[airports_data['month']== month,:]
grouped_airports = airports_data.groupby(['airport_name'])
total_flights_month = grouped_airports['arr_flights'].sum()
airport_lat = grouped_airports["Latitude"].unique()
airport_lng = grouped_airports["Longitude"].unique()
total_df = pd.DataFrame({"total_flights":total_flights_month,"latitude":airport_lat,"longitude":airport_lng})
converted_data = total_df.to_dict('index')
converted_data


{'Atlanta, GA: Hartsfield-Jackson Atlanta International': {'total_flights': 2124645.0,
  'latitude': array([33.6367]),
  'longitude': array([84.4281])},
 "Chicago, IL: Chicago O'Hare International": {'total_flights': 1607393.0,
  'latitude': array([41.9808]),
  'longitude': array([87.9067])},
 'Dallas/Fort Worth, TX: Dallas/Fort Worth International': {'total_flights': 1364793.0,
  'latitude': array([32.8969]),
  'longitude': array([97.0381])},
 'Denver, CO: Denver International': {'total_flights': 1249885.0,
  'latitude': array([39.8617]),
  'longitude': array([104.6731])},
 'Houston, TX: William P Hobby': {'total_flights': 317733.0,
  'latitude': array([29.6456]),
  'longitude': array([95.2789])},
 'Las Vegas, NV: McCarran International': {'total_flights': 818114.0,
  'latitude': array([36.08]),
  'longitude': array([115.1522])},
 'Los Angeles, CA: Los Angeles International': {'total_flights': 1215697.0,
  'latitude': array([33.9425]),
  'longitude': array([118.4072])},
 'Minneapolis,

In [105]:
airports_lat_lng = pd.read_sql("SELECT  airport_name,Latitude,Longitude,month,sum(arr_flights) sum_arr_flights FROM flights_data WHERE airport IN ('ATL', 'DFW', 'SFO', 'ORD', 'DEN', 'LAX', 'PHX', 'HOU', 'LAS', 'MSP') group by airport_name,month,Latitude,Longitude",engine)
airports_lat_lng = airports_lat_lng.set_index('airport_name')
month = int("1")
test_data = airports_lat_lng.loc[airports_lat_lng['month']== month,:]
air_dict = test_data.to_dict('index')
air_dict
# print(airports_lat_lng)

# grouped_airports = airports_data.groupby(['airport_name'],as_index=False)
# total_flights_month = grouped_airports['arr_flights'].sum()
# total_flights_month_df = total_flights_month.to_frame()
# total_flights_month_df.shape
# print(airports_lat_lng)
# print(total_flights_month_df)
#total_flights_month_df.merge(airports_lat_lng)

{'Atlanta, GA: Hartsfield-Jackson Atlanta International': {'Latitude': 33.6367,
  'Longitude': 84.4281,
  'month': 1,
  'sum_arr_flights': 183239.0},
 "Chicago, IL: Chicago O'Hare International": {'Latitude': 41.9808,
  'Longitude': 87.9067,
  'month': 1,
  'sum_arr_flights': 131982.0},
 'Dallas/Fort Worth, TX: Dallas/Fort Worth International': {'Latitude': 32.8969,
  'Longitude': 97.0381,
  'month': 1,
  'sum_arr_flights': 124385.0},
 'Denver, CO: Denver International': {'Latitude': 39.8617,
  'Longitude': 104.6731,
  'month': 1,
  'sum_arr_flights': 105599.0},
 'Houston, TX: William P Hobby': {'Latitude': 29.6456,
  'Longitude': 95.2789,
  'month': 1,
  'sum_arr_flights': 28302.0},
 'Las Vegas, NV: McCarran International': {'Latitude': 36.08,
  'Longitude': 115.1522,
  'month': 1,
  'sum_arr_flights': 71669.0},
 'Los Angeles, CA: Los Angeles International': {'Latitude': 33.9425,
  'Longitude': 118.4072,
  'month': 1,
  'sum_arr_flights': 104873.0},
 'Minneapolis, MN: Minneapolis-St P

In [79]:
airports_data = pd.read_sql("SELECT airport_name,arr_flights FROM flights_data",engine)
airport_grouped = airports_data.groupby(["airport_name"])
top_airports = airport_grouped['arr_flights'].sum()
top_airports = top_airports.sort_values(ascending = False)
topten_airports = top_airports.head(10)
topten_airports

airport_name
Atlanta, GA: Hartsfield-Jackson Atlanta International     2124645.0
Chicago, IL: Chicago O'Hare International                 1607393.0
Dallas/Fort Worth, TX: Dallas/Fort Worth International    1364793.0
Denver, CO: Denver International                          1249885.0
Los Angeles, CA: Los Angeles International                1215697.0
San Francisco, CA: San Francisco International             946224.0
Phoenix, AZ: Phoenix Sky Harbor International              913972.0
Houston, TX: George Bush Intercontinental/Houston          881561.0
Las Vegas, NV: McCarran International                      818114.0
Minneapolis, MN: Minneapolis-St Paul International         731826.0
Name: arr_flights, dtype: float64